In [ ]:
!pip install spotipy
!pip install wheel
!pip install pandas
!pip install psycopg2

In [ ]:
with open("C:/Users/Usuario/Documents/client_secret_spotify.txt",'r') as f:
    pwd= f.read()
print (pwd)

In [7]:
import spotipy
#import pandas as pd
import json
from spotipy.oauth2 import SpotifyClientCredentials
client_id = '65a8d201a0fb46a095fa066b70f08123'
client_secret = pwd
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [76]:
import pandas as pd
results = sp.search(q='year:2023', type='track', limit=50)
data = {'Id': [],'Artista': [], 'Cancion': [], 'Genero': [],'Album': [], 'Total_canciones_album': [], 'Reproducciones': [], 'Anio': []}
for track in results['tracks']['items']:
    id = track['id']
    artist_name = track['artists'][0]['name']
    artist_id = track['artists'][0]['id']
    track_name = track['name']
    track_id = track['id']
    album_group = track['album']['name']
    album_cont = track['album']['total_tracks']
    track_genre = sp.artist(artist_id)['genres']
    track_popularity = track['popularity']
    track_year = track['album']['release_date'][:4]
    
    track_name = track_name.replace("'", "")
    album_group = album_group.replace("'", "")
    track_genre = ', '.join(track_genre)

    data['Id'].append(id)
    data['Artista'].append(artist_name)
    data['Cancion'].append(track_name)
    data['Album'].append(album_group)
    data['Total_canciones_album'].append(album_cont)
    data['Genero'].append(track_genre)
    data['Reproducciones'].append(track_popularity)
    data['Anio'].append(track_year)
df = pd.DataFrame(data)
display(df)

,Id,Artista,Cancion,Genero,Album,Total_canciones_album,Reproducciones,Anio
0,6AQbmUe0Qwf5PZnt4HmTXv,PinkPantheress,Boys a liar Pt. 2,,Boys a liar Pt. 2,2,97,2023
1,0yLdNVWF3Srea0uzk55zFn,Miley Cyrus,Flowers,pop,Flowers,1,100,2023
2,59uQI0PADDKeE6UZDTJEe8,Morgan Wallen,Last Night,contemporary country,3 Songs At A Time Sampler,3,89,2023
3,7K3BhSpAxZBznislvUMVtn,Morgan Wallen,Last Night,contemporary country,One Thing At A Time,36,88,2023
4,0PAcdVzhPO4gq1Iym9ESnK,Morgan Wallen,Thinkin’ Bout Me,contemporary country,One Thing At A Time,36,86,2023
5,7oDd86yk8itslrA9HRP2ki,The Weeknd,Die For You - Remix,"canadian contemporary r&b, canadian pop, pop",Die For You (Remix),1,95,2023
6,0DWdj2oZMBFSzRsi2Cvfzf,KAROL G,TQG,"reggaeton, reggaeton colombiano, urbano latino",MAÑANA SERÁ BONITO,17,98,2023
7,0FA4wrjDJvJTTU8AepZTup,Lil Uzi Vert,Watch This - ARIZONATEARS Pluggnb Remix,"melodic rap, philly rap, rap, trap",Watch This (ARIZONATEARS Pluggnb Remix),1,90,2023
8,3dnP0JxCgygwQH9Gm7q7nb,Eslabon Armado,Ella Baila Sola,"corrido, corridos tumbados, nueva musica mexic...",Ella Baila Sola,1,94,2023
9,2YVlDlk2zRceefo6XlC6DX,Morgan Wallen,Ain’t That Some,contemporary country,One Thing At A Time,36,84,2023


In [5]:
import psycopg2
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user="glo_tarcaya_coderhouse"
with open("C:/Users/Usuario/Documents/pwd_redshift.txt",'r') as f:
    pwd= f.read()
try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Connected to Redshift successfully!")
    
except Exception as e:
    print("Unable to connect to Redshift.")
    print(e)

Connected to Redshift successfully!


In [ ]:
cur = conn.cursor()
# Execute a SQL query to select data from a table
cur.execute("SELECT * FROM prueba")
# Fetch the results
results = cur.fetchall()
results

In [78]:
from psycopg2.extras import execute_values
cur = conn.cursor()

for track in results['tracks']['items']:
    id = track['id']
    artist_name = track['artists'][0]['name']
    artist_id = track['artists'][0]['id']
    track_name = track['name']
    track_id = track['id']
    album_group = track['album']['name']
    album_cont = track['album']['total_tracks']
    track_genre = sp.artist(artist_id)['genres']
    track_popularity = track['popularity']
    track_year = track['album']['release_date'][:4]
    
    track_name = track_name.replace("'", "")
    album_group = album_group.replace("'", "")
    track_genre = ', '.join(track_genre)
    cur.execute(f"INSERT INTO prueba (Id, Artista, Cancion,Genero,Album,Total_canciones_album,Reproducciones,Anio) VALUES ('{id}','{artist_name}', '{track_name}', '{track_genre}', '{ album_group}', {album_cont}, {track_popularity}, {track_year})")

    conn.commit()


In [7]:
cur.close()
conn.close()